In [1]:
import os
import numpy as np
import random
from collections import Counter
from PIL import Image
# from google.colab import files 
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential, Model, clone_model      
from keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Input, ConvLSTM2D,MaxPooling3D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, Callback, LearningRateScheduler
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, InceptionV3, DenseNet121, EfficientNetB2
from tensorflow.keras.layers import LSTM, GRU, TimeDistributed, Input
from tensorflow.keras.models import load_model
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import shutil

#  Spectrogram

In [2]:
data_dir = 'F:/Abdipour/Spectrogram Images/jpg/Temporal/'

def extract_numeric_part(folder_name):
    parts = folder_name.split('_')
    for part in parts:
        if part.startswith('subject'):
            return int(part.replace('subject', ''))
    return -1  # Return -1 if no numeric part is found

folders = sorted([f for f in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, f))], key=extract_numeric_part)

subject_folders, labels = [], []

for folder_name in folders:
    subject_num = extract_numeric_part(folder_name)
    subject_folder = os.path.join(data_dir, folder_name)
    subject_folders.append(subject_folder)

    if subject_num <= 36:
        labels.append(0)  # AD group
    elif 37 <= subject_num <= 65:
        labels.append(1)  # HC group

subject_folders = np.array(subject_folders)
labels = np.array(labels)

print("Subject Folders:", subject_folders)
print("Labels:", labels)

Subject Folders: ['F:/Abdipour/Spectrogram Images/jpg/Temporal/subject1'
 'F:/Abdipour/Spectrogram Images/jpg/Temporal/subject2'
 'F:/Abdipour/Spectrogram Images/jpg/Temporal/subject3'
 'F:/Abdipour/Spectrogram Images/jpg/Temporal/subject4'
 'F:/Abdipour/Spectrogram Images/jpg/Temporal/subject5'
 'F:/Abdipour/Spectrogram Images/jpg/Temporal/subject6'
 'F:/Abdipour/Spectrogram Images/jpg/Temporal/subject7'
 'F:/Abdipour/Spectrogram Images/jpg/Temporal/subject8'
 'F:/Abdipour/Spectrogram Images/jpg/Temporal/subject9'
 'F:/Abdipour/Spectrogram Images/jpg/Temporal/subject10'
 'F:/Abdipour/Spectrogram Images/jpg/Temporal/subject11'
 'F:/Abdipour/Spectrogram Images/jpg/Temporal/subject12'
 'F:/Abdipour/Spectrogram Images/jpg/Temporal/subject13'
 'F:/Abdipour/Spectrogram Images/jpg/Temporal/subject14'
 'F:/Abdipour/Spectrogram Images/jpg/Temporal/subject15'
 'F:/Abdipour/Spectrogram Images/jpg/Temporal/subject16'
 'F:/Abdipour/Spectrogram Images/jpg/Temporal/subject17'
 'F:/Abdipour/Spectrogr

In [3]:
def load_images_and_labels(subjects, labels):
    images, labels_t = [], []
    for subject_folder, label in zip(subjects, labels):
        for filename in os.listdir(subject_folder):
            img_path = os.path.join(subject_folder, filename)
            try:
                img = Image.open(img_path)
                img = img.resize((224, 224))
                img_array = np.array(img, dtype=np.float32)
                img_array = img_array / 255.0  
                images.append(img_array)
                labels_t.append(label)
            except Exception as e:
                print(f"Error loading image: {img_path}, {e}")

    return np.array(images), np.array(labels_t)

In [4]:
def load_and_preprocess_data(subject_folders, labels):
    train_subjects, test_subjects, train_labels, test_labels = train_test_split(subject_folders, labels, test_size=0.2, stratify=labels)

    train_subjects, val_subjects, train_labels, val_labels = train_test_split(train_subjects, train_labels, test_size=0.1, stratify=train_labels)

    train_images, train_labels_t = load_images_and_labels(train_subjects, train_labels)
    val_images, val_labels_t = load_images_and_labels(val_subjects, val_labels)

    test_image_counts_per_subject = [len(os.listdir(folder)) for folder in test_subjects]

    test_images, test_labels_t = load_images_and_labels(test_subjects, test_labels)

    return train_images, train_labels_t, val_images, val_labels_t, test_images, test_labels_t, test_subjects, test_image_counts_per_subject



In [5]:
def create_new_model(base_model):
    model = clone_model(base_model)

    for layer in model.layers:
        layer.trainable = False

    x = model.output
    x = Flatten()(x)
    x = Dense(128)(x)
    x = Dropout(0.2)(x)
    x = Dense(256)(x)
    x = Dropout(0.5)(x)
    x = Dense(512)(x)
    x = Dropout(0.2)(x)

    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=model.input, outputs=predictions)
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model

In [6]:
def train_and_evaluate_model(base_model, subject_folders, labels, num_epochs=50, num_iterations=5, model_name='model'):
    accuracy_list, precision_list, recall_list, f1_list = [], [], [], []

    for iteration in range(num_iterations):
        print(f"\nIteration {iteration + 1}:")

        model = create_new_model(base_model)

        train_images, train_labels_t, val_images, val_labels_t, test_images, test_labels_t, test_subjects, test_image_counts_per_subject = load_and_preprocess_data(subject_folders, labels)

        train_data = list(zip(train_images, train_labels_t))
        np.random.shuffle(train_data)
        train_images, train_labels_t = zip(*train_data)

        val_data = list(zip(val_images, val_labels_t))
        np.random.shuffle(val_data)
        val_images, val_labels_t = zip(*val_data)

        # test_data = list(zip(test_images, test_labels_t))
        # np.random.shuffle(test_data)
        # test_images, test_labels_t = zip(*test_data)

        train_images, train_labels_t = np.array(train_images), np.array(train_labels_t)
        val_images, val_labels_t = np.array(val_images), np.array(val_labels_t)
        # test_images, test_labels_t = np.array(test_images), np.array(test_labels_t)



        # checkpoint_filepath = "best_weights_iteration_{iteration + 1}.h5"
        # checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,
        #                                        monitor='val_accuracy',
        #                                        mode='max',
        #                                        save_best_only=True,
        #                                        save_weights_only=True,
        #                                        verbose=1)

        # history = model.fit(train_images, train_labels_t, epochs=num_epochs, batch_size=8, validation_data=(val_images, val_labels_t), callbacks=[checkpoint], verbose=1)

        checkpoint_dir = 'F:/Abdipour/Spectrogram Images/'
        os.makedirs(checkpoint_dir, exist_ok=True)

        # local_checkpoint_filepath = os.path.join(checkpoint_dir, f"{model_name}_best_weights_iteration_{iteration + 1}.h5")
        # drive_checkpoint_filepath = f'/content/drive/My Drive/{model_name}_best_weights_iteration_{iteration + 1}.h5'
        local_checkpoint_filepath = checkpoint_dir + '\\' + f"{model_name}_best_weights_iteration_{iteration +1}.weights.h5"

        checkpoint_callback = ModelCheckpoint(filepath=local_checkpoint_filepath,
                                               monitor='val_accuracy',
                                               mode='max',
                                               save_best_only=True,
                                               save_weights_only=True,
                                               verbose=1)

        # class CustomEarlyStopping(EarlyStopping):
        #     def on_epoch_end(self, epoch, logs=None):
        #         if epoch > 30:
        #             super().on_epoch_end(epoch - 30, logs)

        class LearningRateLogger(Callback):
            def on_epoch_end(self, epoch, logs=None):
                logs = logs or {}
                logs['learning_rate'] = self.model.optimizer.learning_rate.numpy()
        lr_logger = LearningRateLogger()
        # initial_epochs = 30        
        # history = model.fit(train_images, train_labels_t, epochs=initial_epochs, batch_size=16, validation_data=(val_images, val_labels_t), verbose=1)

        # early_stopping = CustomEarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', baseline=None, restore_best_weights=False, start_from_epoch=30)
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', baseline=None, restore_best_weights=False)
        # history = model.fit(train_images, train_labels_t, epochs=num_epochs, batch_size=16, validation_data=(val_images, val_labels_t), callbacks=[checkpoint_callback, early_stopping], initial_epoch=initial_epochs, verbose=1)
        history = model.fit(train_images, train_labels_t, epochs=num_epochs, batch_size=8, validation_data=(val_images, val_labels_t), callbacks=[checkpoint_callback, early_stopping], verbose=1)

        # if os.path.exists(local_checkpoint_filepath):
        #     print(f"Best weights saved locally at: {local_checkpoint_filepath}")

        #     shutil.copy(local_checkpoint_filepath, drive_checkpoint_filepath)
        #     if os.path.exists(drive_checkpoint_filepath):
        #         print(f"Best weights also copied to Google Drive at: {drive_checkpoint_filepath}")
        #     else:
        #         print(f"Failed to copy best weights to Google Drive at: {drive_checkpoint_filepath}")
        # else:
        #     print(f"Failed to save best weights locally at: {local_checkpoint_filepath}")

        model.load_weights(local_checkpoint_filepath)

        test_predictions = model.predict(test_images)
        test_predictions = (test_predictions > 0.5).astype(int)

        accuracy = accuracy_score(test_labels_t, test_predictions)
        precision = precision_score(test_labels_t, test_predictions)
        recall = recall_score(test_labels_t, test_predictions)
        f1 = f1_score(test_labels_t, test_predictions)

        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)

        print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

        print("ModelCheckpoint callback triggered and saved the best weights.")


        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('Model accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        file_name = f'{model_name}_accuracy_iteration_{iteration + 1}.png'
        plt.savefig(file_name)
        plt.close()
        # files.download(file_name)



        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        file_name = f'{model_name}_loss_iteration_{iteration + 1}.png'
        plt.savefig(file_name)
        plt.close()
        # files.download(file_name)


        cm = confusion_matrix(test_labels_t, test_predictions)
        plt.figure(figsize=(6, 5))
        sns.heatmap(cm, annot=True, fmt='d', xticklabels=['AD', 'HC'], yticklabels=['AD', 'HC'])
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title(f'Confusion Matrix - {model_name} - Iteration {iteration + 1}')
        file_name = f'{model_name}_confusion_matrix_iteration_{iteration + 1}.png'
        plt.savefig(file_name)
        plt.close()
        # files.download(file_name)




        # print("\nPredictions for Each Subject's Images:")
        # start_idx = 0
        # for i, (subject_folder, num_images) in enumerate(zip(test_subjects, test_image_counts_per_subject)):
        #     end_idx = start_idx + num_images
        #     subject_images = test_images[start_idx:end_idx]
        #     subject_predictions = (model.predict(subject_images) > 0.5).astype(int)
        #     print(f"Subject {i + 1} ({subject_folder}):")
        #     for j, prediction in enumerate(subject_predictions):
        #         if prediction == 0:
        #             print(f"Image {j + 1}: AD")
        #         else:
        #             print(f"Image {j + 1}: HC")
        #     start_idx = end_idx


        print("\nPredictions for Each Subject's Images:")
        start_idx = 0
        for i, (subject_folder, num_images) in enumerate(zip(test_subjects, test_image_counts_per_subject)):
            end_idx = start_idx + num_images
            subject_images = test_images[start_idx:end_idx]
            subject_predictions = (model.predict(subject_images) > 0.5).astype(int)
            ad_count = sum(subject_predictions == 0)
            hc_count = sum(subject_predictions == 1)
            print(f"Subject {i + 1} ({subject_folder}):")
            print(f"AD count: {ad_count}, HC count: {hc_count}")
            start_idx = end_idx




    print("\nAverage Metrics:")
    print(f"Average Accuracy: {np.mean(accuracy_list):.4f}")
    print(f"Average Precision: {np.mean(precision_list):.4f}")
    print(f"Average Recall: {np.mean(recall_list):.4f}")
    print(f"Average F1 Score: {np.mean(f1_list):.4f}")



# Occipital 

## VGG16

In [7]:
vgg16_base_model = VGG16(weights ='imagenet', include_top =False, input_shape = (224,224,3))

In [8]:
model_name = 'vgg16_Occipital_Spectrogram'
train_and_evaluate_model(vgg16_base_model, subject_folders, labels, num_epochs=100, num_iterations=1, model_name=model_name)


Iteration 1:
Epoch 1/100
229/230 [============================>.] - ETA: 0s - loss: 0.7042 - accuracy: 0.5314
Epoch 1: val_accuracy improved from -inf to 0.54589, saving model to F:/Abdipour/Spectrogram Images\vgg16_Occipital_Spectrogram_best_weights_iteration_1.weights.h5
230/230 [==============================] - 18s 58ms/step - loss: 0.7043 - accuracy: 0.5310 - val_loss: 0.6881 - val_accuracy: 0.5459 - learning_rate: 0.0010
Epoch 2/100
230/230 [==============================] - ETA: 0s - loss: 0.6833 - accuracy: 0.5625
Epoch 2: val_accuracy did not improve from 0.54589
230/230 [==============================] - 11s 47ms/step - loss: 0.6833 - accuracy: 0.5625 - val_loss: 0.6874 - val_accuracy: 0.5459 - learning_rate: 0.0010
Epoch 3/100
229/230 [============================>.] - ETA: 0s - loss: 0.6513 - accuracy: 0.6269
Epoch 3: val_accuracy improved from 0.54589 to 0.59420, saving model to F:/Abdipour/Spectrogram Images\vgg16_Occipital_Spectrogram_best_weights_iteration_1.weights.h5

In [8]:
model_name = 'vgg16_Occipital_Spectrogram'
train_and_evaluate_model(vgg16_base_model, subject_folders, labels, num_epochs=100, num_iterations=1, model_name=model_name)


Iteration 1:
Epoch 1/100
224/224 [==============================] - ETA: 0s - loss: 0.7051 - accuracy: 0.5179
Epoch 1: val_accuracy improved from -inf to 0.55708, saving model to F:/Abdipour/Spectrogram Images\vgg16_Occipital_Spectrogram_best_weights_iteration_1.weights.h5
224/224 [==============================] - 18s 61ms/step - loss: 0.7051 - accuracy: 0.5179 - val_loss: 0.6864 - val_accuracy: 0.5571
Epoch 2/100
223/224 [============================>.] - ETA: 0s - loss: 0.6937 - accuracy: 0.5398
Epoch 2: val_accuracy did not improve from 0.55708
224/224 [==============================] - 11s 47ms/step - loss: 0.6937 - accuracy: 0.5397 - val_loss: 0.6840 - val_accuracy: 0.5571
Epoch 3/100
223/224 [============================>.] - ETA: 0s - loss: 0.6798 - accuracy: 0.5788
Epoch 3: val_accuracy improved from 0.55708 to 0.56164, saving model to F:/Abdipour/Spectrogram Images\vgg16_Occipital_Spectrogram_best_weights_iteration_1.weights.h5
224/224 [==============================] - 11s 

# Parietal

## ResNet50

In [ ]:
ResNet50_base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
model_name = 'ResNet50_Parietal_Spectrogram'
train_and_evaluate_model(ResNet50_base_model, subject_folders, labels, num_epochs=100, num_iterations=1, model_name=model_name)

## VGG16

In [7]:
vgg16_base_model = VGG16(weights ='imagenet', include_top =False, input_shape = (224,224,3))

In [8]:
model_name = 'vgg16_Parietal_Spectrogram'
train_and_evaluate_model(vgg16_base_model, subject_folders, labels, num_epochs=100, num_iterations=1, model_name=model_name)


Iteration 1:
Error loading image: F:/Abdipour/Spectrogram Images/jpg/Parietal/subject1\subject1_channel19_segment26_spectrogram.jpg, cannot identify image file 'F:\\Abdipour\\Spectrogram Images\\jpg\\Parietal\\subject1\\subject1_channel19_segment26_spectrogram.jpg'
Epoch 1/100
664/664 [==============================] - ETA: 0s - loss: 0.6952 - accuracy: 0.5552
Epoch 1: val_accuracy improved from -inf to 0.36502, saving model to F:/Abdipour/Spectrogram Images\vgg16_Parietal_Spectrogram_best_weights_iteration_1.weights.h5
664/664 [==============================] - 27s 34ms/step - loss: 0.6952 - accuracy: 0.5552 - val_loss: 0.7180 - val_accuracy: 0.3650
Epoch 2/100
663/664 [============================>.] - ETA: 0s - loss: 0.6779 - accuracy: 0.5752
Epoch 2: val_accuracy improved from 0.36502 to 0.68568, saving model to F:/Abdipour/Spectrogram Images\vgg16_Parietal_Spectrogram_best_weights_iteration_1.weights.h5
664/664 [==============================] - 21s 31ms/step - loss: 0.6779 - acc

ResourceExhaustedError: Graph execution error:

Detected at node 'model/block1_conv1/Relu' defined at (most recent call last):
    File "c:\Users\BioElectric\anaconda3\envs\AD\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\BioElectric\anaconda3\envs\AD\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\BioElectric\anaconda3\envs\AD\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "c:\Users\BioElectric\anaconda3\envs\AD\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "c:\Users\BioElectric\anaconda3\envs\AD\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "c:\Users\BioElectric\anaconda3\envs\AD\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\BioElec

## InceptionV3

In [ ]:
inception_base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

87910968/87910968 [==============================] - 0s 0us/step


# Temporal

## ResNet50


In [7]:
ResNet50_base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [8]:
model_name = 'ResNet50_Temporal_Spectrogram'
train_and_evaluate_model(ResNet50_base_model, subject_folders, labels, num_epochs=100, num_iterations=1, model_name=model_name)


Iteration 1:
Epoch 1/100
917/917 [==============================] - ETA: 0s - loss: 2.1067 - accuracy: 0.5106
Epoch 1: val_accuracy improved from -inf to 0.48571, saving model to F:/Abdipour/Spectrogram Images\ResNet50_Temporal_Spectrogram_best_weights_iteration_1.weights.h5
917/917 [==============================] - 33s 31ms/step - loss: 2.1067 - accuracy: 0.5106 - val_loss: 0.6956 - val_accuracy: 0.4857
Epoch 2/100
916/917 [============================>.] - ETA: 0s - loss: 0.8206 - accuracy: 0.5177
Epoch 2: val_accuracy improved from 0.48571 to 0.51429, saving model to F:/Abdipour/Spectrogram Images\ResNet50_Temporal_Spectrogram_best_weights_iteration_1.weights.h5
917/917 [==============================] - 27s 30ms/step - loss: 0.8203 - accuracy: 0.5180 - val_loss: 0.7013 - val_accuracy: 0.5143
Epoch 3/100
915/917 [============================>.] - ETA: 0s - loss: 0.7481 - accuracy: 0.5254
Epoch 3: val_accuracy did not improve from 0.51429
917/917 [==============================] - 

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

## VGG16

In [8]:
vgg16_base_model = VGG16(weights ='imagenet', include_top =False, input_shape = (224,224,3))

In [9]:
model_name = 'vgg16_Temporal_Spectrogram'
train_and_evaluate_model(vgg16_base_model, subject_folders, labels, num_epochs=100, num_iterations=1, model_name=model_name)


Iteration 1:
Epoch 1/100
904/904 [==============================] - ETA: 0s - loss: 0.7150 - accuracy: 0.5149
Epoch 1: val_accuracy improved from -inf to 0.46087, saving model to F:/Abdipour/Spectrogram Images\vgg16_Temporal_Spectrogram_best_weights_iteration_1.weights.h5
904/904 [==============================] - 33s 33ms/step - loss: 0.7150 - accuracy: 0.5149 - val_loss: 0.7442 - val_accuracy: 0.4609
Epoch 2/100
902/904 [============================>.] - ETA: 0s - loss: 0.6934 - accuracy: 0.5334
Epoch 2: val_accuracy did not improve from 0.46087
904/904 [==============================] - 26s 29ms/step - loss: 0.6935 - accuracy: 0.5333 - val_loss: 0.7153 - val_accuracy: 0.4609
Epoch 3/100
903/904 [============================>.] - ETA: 0s - loss: 0.6919 - accuracy: 0.5490
Epoch 3: val_accuracy did not improve from 0.46087
904/904 [==============================] - 27s 30ms/step - loss: 0.6919 - accuracy: 0.5490 - val_loss: 0.7029 - val_accuracy: 0.4609
Epoch 4/100
903/904 [==========

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

## InceptionV3

In [10]:
inception_base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/inception_v3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5: 403 -- Forbidden

In [ ]:
model_name = 'InceptionV3_Temporal'
train_and_evaluate_model(Inception_base_model, subject_folders, labels, num_epochs=100, num_iterations=5, model_name=model_name)

# Central

## ResNet50


In [7]:
ResNet50_base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [8]:
model_name = 'ResNet50_Central'
train_and_evaluate_model(ResNet50_base_model, subject_folders, labels, num_epochs=100, num_iterations=1, model_name=model_name)


Iteration 1:
Epoch 1/100
723/723 [==============================] - ETA: 0s - loss: 2.2410 - accuracy: 0.5112
Epoch 1: val_accuracy improved from -inf to 0.50327, saving model to F:/Abdipour/Spectrogram Images\ResNet50_Central_best_weights_iteration_1.weights.h5
723/723 [==============================] - 25s 29ms/step - loss: 2.2410 - accuracy: 0.5112 - val_loss: 1.2140 - val_accuracy: 0.5033
Epoch 2/100
723/723 [==============================] - ETA: 0s - loss: 0.8960 - accuracy: 0.5090
Epoch 2: val_accuracy did not improve from 0.50327
723/723 [==============================] - 19s 26ms/step - loss: 0.8960 - accuracy: 0.5090 - val_loss: 0.6973 - val_accuracy: 0.5033
Epoch 3/100
723/723 [==============================] - ETA: 0s - loss: 0.7619 - accuracy: 0.5069
Epoch 3: val_accuracy did not improve from 0.50327
723/723 [==============================] - 19s 26ms/step - loss: 0.7619 - accuracy: 0.5069 - val_loss: 0.6885 - val_accuracy: 0.5033
Epoch 4/100
721/723 [====================

## VGG16

In [9]:
vgg16_base_model = VGG16(weights ='imagenet', include_top =False, input_shape = (224,224,3))

In [10]:
model_name = 'vgg16_Temporal_Spectrogram'
train_and_evaluate_model(vgg16_base_model, subject_folders, labels, num_epochs=100, num_iterations=1, model_name=model_name)


Iteration 1:
Epoch 1/100
708/708 [==============================] - ETA: 0s - loss: 0.6968 - accuracy: 0.5448
Epoch 1: val_accuracy improved from -inf to 0.57214, saving model to F:/Abdipour/Spectrogram Images\vgg16_Temporal_Spectrogram_best_weights_iteration_1.weights.h5
708/708 [==============================] - 23s 30ms/step - loss: 0.6968 - accuracy: 0.5448 - val_loss: 0.6869 - val_accuracy: 0.5721
Epoch 2/100
708/708 [==============================] - ETA: 0s - loss: 0.6862 - accuracy: 0.5607
Epoch 2: val_accuracy did not improve from 0.57214
708/708 [==============================] - 20s 29ms/step - loss: 0.6862 - accuracy: 0.5607 - val_loss: 0.7217 - val_accuracy: 0.4279
Epoch 3/100
707/708 [============================>.] - ETA: 0s - loss: 0.6869 - accuracy: 0.5592
Epoch 3: val_accuracy did not improve from 0.57214
708/708 [==============================] - 20s 29ms/step - loss: 0.6869 - accuracy: 0.5591 - val_loss: 0.6766 - val_accuracy: 0.5721
Epoch 4/100
708/708 [==========

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

## InceptionV3

In [ ]:
inception_base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/inception_v3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5: 403 -- Forbidden

In [ ]:
model_name = 'InceptionV3_Temporal'
train_and_evaluate_model(Inception_base_model, subject_folders, labels, num_epochs=100, num_iterations=5, model_name=model_name)